The paper aims to perform Bayesian inference for the weights $w$ of a neural network. This involves finding the posterior distribution $P(w|D)$ given the data $D$. Since calculating the true posterior is often intractable, they use Variational Inference (VI). In VI, we approximate the true posterior $P(w|D)$ with a simpler, parameterized distribution $q(w|\theta)$, called the variational posterior. The goal is to find the parameters $\theta$ that make $q(w|\theta)$ as close as possible to $P(w|D)$, typically by minimizing the KL divergence, $\text{KL}[q(w|\theta) || P(w|D)]$, which is equivalent to maximizing the Evidence Lower Bound (ELBO) or minimizing the Variational Free Energy (given in Eq. 1 of the paper).

The objective function (Variational Free Energy / negative ELBO) is:
$$ F(D, \theta) = \text{KL}[q(w|\theta) || P(w)] - \mathbb{E}_{q(w|\theta)}[\log P(D|w)] $$
This can be rewritten as an expectation:
$$ F(D, \theta) = \mathbb{E}_{q(w|\theta)} [ \log q(w|\theta) - \log P(w) - \log P(D|w) ] $$
Let $f(w, \theta) = \log q(w|\theta) - \log P(w) - \log P(D|w)$. Then $F(D, \theta) = \mathbb{E}_{q(w|\theta)} [f(w, \theta)]$.

To minimize $F(D, \theta)$ using gradient descent, we need to compute its gradient with respect to $\theta$:
$$ \nabla_\theta F(D, \theta) = \nabla_\theta \mathbb{E}_{q(w|\theta)} [f(w, \theta)] $$

**The Problem:**

The challenge is that the expectation $\mathbb{E}_{q(w|\theta)}$ is taken over $q(w|\theta)$, which *itself* depends on the parameters $\theta$ we are differentiating with respect to. You cannot simply move the gradient inside the expectation like $\mathbb{E}_{q(w|\theta)} [\nabla_\theta f(w, \theta)]$ because the distribution changes with $\theta$.

**The Solution (Reparameterization Trick - Proposition 1):**

Section 3.1 proposes a way to get an *unbiased* estimate of this gradient using Monte Carlo sampling, provided we can reparameterize the sampling process.

**Proposition 1 Explained:**

1.  **Base Distribution:** Assume there's a random variable $\epsilon$ that comes from a simple, fixed probability distribution $q(\epsilon)$ which *does not* depend on $\theta$. Common choices are standard Gaussian $\mathcal{N}(0, I)$ or Uniform $\mathcal{U}(0, 1)$.
2.  **Deterministic Transformation:** Assume we can express the random variable $w$ (which follows $q(w|\theta)$) as a *deterministic* and *differentiable* function $t$ of the parameters $\theta$ and the base noise $\epsilon$. That is: $w = t(\theta, \epsilon)$.
3.  **Change of Variables:** The condition $q(\epsilon)d\epsilon = q(w|\theta)dw$ just formally states that sampling $\epsilon$ from $q(\epsilon)$ and transforming it via $w = t(\theta, \epsilon)$ is equivalent to sampling $w$ directly from $q(w|\theta)$.
4.  **The Result:** Under these conditions, the gradient of the expectation can be rewritten as an expectation over the *fixed* distribution $q(\epsilon)$:
    $$ \frac{\partial}{\partial\theta} \mathbb{E}_{q(w|\theta)} [f(w, \theta)] = \mathbb{E}_{q(\epsilon)} \left[ \frac{\partial}{\partial\theta} f(t(\theta, \epsilon), \theta) \right] $$
    Since $q(\epsilon)$ doesn't depend on $\theta$, we moved the derivative inside. Now, we apply the chain rule to differentiate $f(t(\theta, \epsilon), \theta)$ with respect to $\theta$. $f$ depends on $\theta$ both directly and indirectly through $w = t(\theta, \epsilon)$:
    $$ \frac{\partial}{\partial\theta} f(t(\theta, \epsilon), \theta) = \frac{\partial f}{\partial w} \frac{\partial w}{\partial \theta} + \frac{\partial f}{\partial \theta} $$
    Substituting $w = t(\theta, \epsilon)$ and $\frac{\partial w}{\partial \theta} = \frac{\partial t(\theta, \epsilon)}{\partial \theta}$, we get:
    $$ \frac{\partial}{\partial\theta} f(t(\theta, \epsilon), \theta) = \left. \frac{\partial f}{\partial w} \right|_{w=t(\theta, \epsilon)} \frac{\partial t(\theta, \epsilon)}{\partial \theta} + \left. \frac{\partial f}{\partial \theta} \right|_{w=t(\theta, \epsilon)} $$
    Putting it back into the expectation:
    $$ \frac{\partial}{\partial\theta} \mathbb{E}_{q(w|\theta)} [f(w, \theta)] = \mathbb{E}_{q(\epsilon)} \left[ \left. \frac{\partial f}{\partial w} \right|_{w=t(\theta, \epsilon)} \frac{\partial t(\theta, \epsilon)}{\partial \theta} + \left. \frac{\partial f}{\partial \theta} \right|_{w=t(\theta, \epsilon)} \right] $$
    This is exactly the result stated in Proposition 1.

**Why is this useful?**

We can now estimate the gradient using Monte Carlo:
1.  Sample $M$ values $\epsilon^{(i)}$ from the fixed base distribution $q(\epsilon)$ for $i = 1, \dots, M$.
2.  For each $\epsilon^{(i)}$, compute $w^{(i)} = t(\theta, \epsilon^{(i)})$.
3.  For each pair $(w^{(i)}, \epsilon^{(i)})$, compute the term inside the expectation:
    $$ g^{(i)} = \left[ \left. \frac{\partial f}{\partial w} \right|_{w=w^{(i)}} \frac{\partial t(\theta, \epsilon^{(i)})}{\partial \theta} + \left. \frac{\partial f}{\partial \theta} \right|_{w=w^{(i)}} \right] $$
4.  Average these terms to get the gradient estimate:
    $$ \nabla_\theta \mathbb{E}_{q(w|\theta)} [f(w, \theta)] \approx \hat{g} = \frac{1}{M} \sum_{i=1}^M g^{(i)} $$

This estimator $\hat{g}$ is unbiased ($\mathbb{E}[\hat{g}] = \nabla_\theta F(D, \theta)$) and often has much lower variance than alternatives like the score function estimator (REINFORCE), making optimization more stable.

**Simple Step-by-Step Implementation (Conceptual)**

Let's assume $q(w_j|\theta_j)$ is a Gaussian distribution for each weight $w_j$, parameterized by $\theta_j = (\mu_j, \rho_j)$, where the standard deviation is $\sigma_j = \log(1 + \exp(\rho_j))$ (softplus function to ensure positivity). The transformation is $w_j = t(\theta_j, \epsilon_j) = \mu_j + \sigma_j \epsilon_j = \mu_j + \log(1 + \exp(\rho_j))\epsilon_j$, where $\epsilon_j \sim \mathcal{N}(0, 1)$.

In [1]:
import numpy as np

# --- Define Distributions and Functions ---

def softplus(x):
  """Ensures standard deviation is positive."""
  return np.log1p(np.exp(x)) # log(1 + exp(x))

def q_epsilon(shape):
  """Sample from the base distribution q(epsilon) = N(0, 1)."""
  return np.random.randn(*shape)

def transform_t(mu, rho, epsilon):
  """Deterministic transform: w = t(theta, epsilon) = mu + sigma * epsilon"""
  sigma = softplus(rho)
  return mu + sigma * epsilon

def log_q_w_theta(w, mu, rho):
  """log probability density of q(w|theta) = N(w | mu, sigma^2)"""
  sigma = softplus(rho)
  return -0.5 * np.log(2 * np.pi) - np.log(sigma) - 0.5 * ((w - mu) / sigma)**2

def log_prior_P_w(w):
  """log probability of the prior P(w). Example: Standard Gaussian N(0,1)"""
  # Assumes a simple prior for demonstration, might be more complex.
  return -0.5 * np.log(2 * np.pi) - 0.5 * w**2

def log_likelihood_P_Dw(w, data_X, data_y):
  """log probability of data given weights P(D|w). Needs a model forward pass."""
  # This is highly dependent on the neural network architecture and task.
  # Placeholder: Assume a simple linear model prediction and Gaussian likelihood
  # preds = data_X @ w  # Simplified example
  # likelihood_std = 1.0
  # return -0.5 * np.log(2 * np.pi * likelihood_std**2) * len(data_y) \
  #        - 0.5 * np.sum((preds - data_y)**2) / likelihood_std**2
  # --- Replace with actual network forward pass and loss calculation ---
  # For simplicity here, let's return a dummy value or a simple function of w
  # Example: Treat it like another prior for illustration of derivatives
  return -0.5 * (w - 0.5)**2 # Dummy example likelihood term


def f(w, mu, rho, data_X, data_y):
  """The function inside the expectation in Eq. 1 (objective function term)."""
  log_q = log_q_w_theta(w, mu, rho)
  log_P = log_prior_P_w(w)
  log_L = log_likelihood_P_Dw(w, data_X, data_y) # Pass data here
  return log_q - log_P - log_L

# --- Define Gradients (Crucial for Backpropagation) ---
# These would typically be computed automatically by frameworks like
# PyTorch, TensorFlow, JAX using automatic differentiation.
# Here we define them manually for clarity, assuming w, mu, rho are scalars.

def grad_softplus(x):
    """Derivative of softplus."""
    return 1.0 / (1.0 + np.exp(-x)) # Sigmoid function

def grad_t_wrt_mu(mu, rho, epsilon):
    """∂t/∂μ = ∂(μ + σ*ε)/∂μ = 1"""
    return 1.0

def grad_t_wrt_rho(mu, rho, epsilon):
    """∂t/∂ρ = ∂(μ + σ*ε)/∂ρ = (∂σ/∂ρ) * ε"""
    sigma = softplus(rho)
    d_sigma_d_rho = grad_softplus(rho)
    return d_sigma_d_rho * epsilon

def grad_f_wrt_w(w, mu, rho, data_X, data_y):
    """∂f/∂w = ∂(log q - log P - log L)/∂w """
    # Need derivatives of log_q, log_P, log_L w.r.t w
    sigma = softplus(rho)
    d_logq_dw = - (w - mu) / (sigma**2)
    d_logP_dw = - w # For N(0,1) prior example
    d_logL_dw = - (w - 0.5) # For dummy example likelihood
    return d_logq_dw - d_logP_dw - d_logL_dw

def grad_f_wrt_mu(w, mu, rho, data_X, data_y):
   """∂f/∂μ = ∂(log q - log P - log L)/∂μ"""
   # Only log q depends directly on mu
   sigma = softplus(rho)
   d_logq_dmu = (w - mu) / (sigma**2)
   return d_logq_dmu

def grad_f_wrt_rho(w, mu, rho, data_X, data_y):
   """∂f/∂ρ = ∂(log q - log P - log L)/∂ρ"""
   # Only log q depends directly on rho
   sigma = softplus(rho)
   d_sigma_d_rho = grad_softplus(rho)
   d_logq_drho = - (1.0 / sigma) * d_sigma_d_rho + ((w - mu)**2 / sigma**3) * d_sigma_d_rho
   # Term 1: -(d(log sigma)/drho) = -(1/sigma * dsigma/drho)
   # Term 2: -0.5 * d/drho ((w-mu)/sigma)^2 = -0.5 * (w-mu)^2 * d/drho (sigma^-2)
   #         = -0.5 * (w-mu)^2 * (-2 * sigma^-3 * dsigma/drho)
   #         = (w-mu)^2 * sigma^-3 * dsigma/drho
   return d_logq_drho


# --- Monte Carlo Gradient Estimation ---

def estimate_gradient(mu, rho, data_X, data_y, num_samples=1):
    """
    Estimates ∇_θ E[f(w, θ)] using the reparameterization trick.
    Here θ = (mu, rho).
    """
    # Assume mu, rho are numpy arrays of the same shape (e.g., for all weights)
    batch_shape = mu.shape
    total_grad_mu = np.zeros_like(mu)
    total_grad_rho = np.zeros_like(rho)

    for _ in range(num_samples):
        # 1. Sample epsilon ~ N(0, 1)
        epsilon = q_epsilon(batch_shape)

        # 2. Compute w = t(theta, epsilon)
        w = transform_t(mu, rho, epsilon)

        # 3. Compute required gradient components
        df_dw = grad_f_wrt_w(w, mu, rho, data_X, data_y)
        dt_dmu = grad_t_wrt_mu(mu, rho, epsilon) # This is just 1.0
        dt_drho = grad_t_wrt_rho(mu, rho, epsilon)
        df_dmu = grad_f_wrt_mu(w, mu, rho, data_X, data_y) # Direct gradient part
        df_drho = grad_f_wrt_rho(w, mu, rho, data_X, data_y) # Direct gradient part

        # 4. Combine using chain rule from Proposition 1
        # Gradient w.r.t. mu: ∂f/∂w * ∂w/∂μ + ∂f/∂μ
        grad_mu_sample = df_dw * dt_dmu + df_dmu

        # Gradient w.r.t. rho: ∂f/∂w * ∂w/∂ρ + ∂f/∂ρ
        grad_rho_sample = df_dw * dt_drho + df_drho

        # Accumulate
        total_grad_mu += grad_mu_sample
        total_grad_rho += grad_rho_sample

    # 5. Average over samples
    avg_grad_mu = total_grad_mu / num_samples
    avg_grad_rho = total_grad_rho / num_samples

    return avg_grad_mu, avg_grad_rho

# --- Example Usage (within an optimization loop) ---

# Initialize parameters (e.g., for a single weight)
current_mu = np.array([0.0])
current_rho = np.array([-3.0]) # Initial std dev ~ softplus(-3) = 0.047

# Dummy data
dummy_X = np.array([[1.0]])
dummy_y = np.array([1.0])

# Learning rate
learning_rate = 0.01

# Optimization loop (simplified)
for step in range(100):
    # Estimate gradient of the objective F(D, theta)
    # Often use M=1 sample in SGD (Stochastic Gradient Descent)
    grad_mu, grad_rho = estimate_gradient(current_mu, current_rho, dummy_X, dummy_y, num_samples=1)

    # Update parameters (Gradient Descent)
    current_mu -= learning_rate * grad_mu
    current_rho -= learning_rate * grad_rho

    if step % 10 == 0:
        w_sample = transform_t(current_mu, current_rho, q_epsilon(current_mu.shape))
        current_f = f(w_sample, current_mu, current_rho, dummy_X, dummy_y) # Approximate objective
        print(f"Step: {step}, Mu: {current_mu[0]:.4f}, Rho: {current_rho[0]:.4f}, "
              f"Sigma: {softplus(current_rho[0]):.4f}, Approx F: {current_f[0]:.4f}")

Step: 0, Mu: 0.0059, Rho: -2.9905, Sigma: 0.0490, Approx F: 3.0123
Step: 10, Mu: 0.0507, Rho: -2.8932, Sigma: 0.0539, Approx F: 1.5899
Step: 20, Mu: 0.0862, Rho: -2.7962, Sigma: 0.0592, Approx F: 2.2089
Step: 30, Mu: 0.1176, Rho: -2.6999, Sigma: 0.0650, Approx F: 2.5299
Step: 40, Mu: 0.1440, Rho: -2.6049, Sigma: 0.0713, Approx F: 1.3823
Step: 50, Mu: 0.1677, Rho: -2.5099, Sigma: 0.0781, Approx F: 1.9747
Step: 60, Mu: 0.1799, Rho: -2.4152, Sigma: 0.0856, Approx F: 0.9986
Step: 70, Mu: 0.1898, Rho: -2.3201, Sigma: 0.0937, Approx F: 1.8584
Step: 80, Mu: 0.2146, Rho: -2.2286, Sigma: 0.1023, Approx F: 2.3429
Step: 90, Mu: 0.2206, Rho: -2.1350, Sigma: 0.1118, Approx F: 1.3683


**Key Points in Implementation:**

1.  **Reparameterization:** The core is $w = t(\theta, \epsilon)$. Randomness comes *only* from $\epsilon$.
2.  **Gradient Calculation:** The gradient estimate $\hat{g}$ implements the formula from Proposition 1 using the chain rule components $\frac{\partial f}{\partial w}$, $\frac{\partial t}{\partial \theta}$, and $\frac{\partial f}{\partial \theta}$.
3.  **Automatic Differentiation:** In practice (PyTorch, TensorFlow, JAX), you wouldn't compute these partial derivatives manually. You'd define the forward computation ($\epsilon \rightarrow w \rightarrow f$), and the framework would compute the necessary gradients $\nabla_\theta f(t(\theta, \epsilon), \theta)$ via backpropagation. The reparameterization $w = t(\theta, \epsilon)$ ensures the gradient path flows correctly back from $f$ through $w$ to $\theta = (\mu, \rho)$.
4.  **Objective Function Term $f$:** This function $f(w, \theta) = \log q(w|\theta) - \log P(w) - \log P(D|w)$ encapsulates the terms inside the expectation of the variational objective.
5.  **Monte Carlo:** The gradient $\nabla_\theta F(D, \theta)$ is estimated by averaging over one or more samples ($M$). Using $M=1$ corresponds to Stochastic Gradient Descent.

This trick allows optimizing the parameters $\theta$ of the variational distribution $q(w|\theta)$ using standard gradient-based methods, forming the basis of algorithms like Bayes by Backprop.